## Classroom Assignment 1: 
Assesment:
 
Input a prompt from the user
Generate 3 outcomes from a generative AI model:

a. A story of 100 words about the prompt 

b. This story should be represented in a vector/embeddings format 

c. Find all Part-of-speeches in the generated story 
 

## Installing & Importing required libraries

In [0]:
!pip install openai==1.13.3

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for openai==1.13.3 from https://files.pythonhosted.org/packages/a9/cc/f2bbce0ad52e09cd1aecb724af06385021b42a7317cd5938ba9c8581509d/openai-1.13.3-py3-none-any.whl.metadata
  Using cached openai-1.13.3-py3-none-any.whl.metadata (18 kB)
Using cached openai-1.13.3-py3-none-any.whl (227 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.29.0
    Not uninstalling openai at /databricks/python3/lib/python3.11/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-3486f682-8b38-453e-8b2d-7883ae175008
    Can't uninstall 'openai'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
# Install necessary libraries 
%pip install transformers torch datasets

# Import required modules
from transformers import pipeline, AutoTokenizer
import asyncio
from openai import AsyncAzureOpenAI
import spacy

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


2024-11-22 15:40:55.337228: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-22 15:40:55.338036: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-22 15:40:55.341149: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-22 15:40:55.380475: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-22 15:40:56.189758: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [0]:
%sh
python3 -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## Logic

- Story generation: Azure OpenAI
- Embedding generation: Huggingface GPT2
- POS Tagging: Spacy

In [0]:
async def main():
  try:
        # Get configuration settings
        # Configuration settings
        azure_oai_endpoint = ""
        azure_oai_key = ""
        azure_oai_deployment =""

        # Configure the Azure OpenAI client
        client = AsyncAzureOpenAI(
            azure_endpoint = azure_oai_endpoint,
            api_key=azure_oai_key,
            api_version="2024-02-15-preview"
        )

        while True:
            prompt_input = input('Enter the prompt for the story (Press Q to quit):')

            if prompt_input.lower() == 'q':
                print('Exiting program...')
                break

            for _ in range(3):
                # Generate a story
                story = await generate_story(prompt_input, model=azure_oai_deployment, client=client)
                print("Generated Story:\n", story)

                # Get story embeddings
                tokenizer = AutoTokenizer.from_pretrained('gpt2')
                tokenizer.pad_token = tokenizer.eos_token
                embeddings = get_embeddings(story, tokenizer=tokenizer)
                print("\nEmbeddings:\n", embeddings)

                # Find parts of speech
                pos_tags = pos_tagging(story)
                print("\nPart-of-Speech Tags:")
                for token, pos in pos_tags:
                    print(f"{token}: {pos}")
  except Exception as ex:
        print(ex)

async def generate_story(prompt, model, client):
    # Provide a basic user message, and use the prompt content as the user message
    system_message = "You are a expert short story writer. Please generate only a short story in 100 words on the mentioned topic or description."
    user_message = prompt

    # Format and send the request to the model
    messages =[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]

    # Call the Azure OpenAI model
    response = await client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=1000
    )
    # Print the response to the console
    return response.choices[0].message.content

# Function to get embeddings
def get_embeddings(text, tokenizer):
    return tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128
    )

# Function to find parts of speech
def pos_tagging(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    pos_tags = [(token.text, token.pos_) for token in doc]
    return pos_tags

## Outputs

In [0]:
await main()

Enter the prompt for the story (Press Q to quit): Red Mangoes

Generated Story:
 In a small village, there was a magical tree that bore red mangoes. These mangoes were said to grant any wish made upon them. One day, a young girl named Maya discovered the tree. She wished for her sick mother to be healed. Miraculously, her wish came true. News spread, and people from far and wide came to make their wishes. However, Maya noticed a change. The mangoes were gradually turning yellow and losing their magic. Maya realized that the true magic was in the love and care she had shown her mother. With that realization, the tree began to bear red mangoes once more.

Embeddings:
 {'input_ids': [818, 257, 1402, 7404, 11, 612, 373, 257, 10883, 5509, 326, 18631, 2266, 49364, 274, 13, 2312, 49364, 274, 547, 531, 284, 7264, 597, 4601, 925, 2402, 606, 13, 1881, 1110, 11, 257, 1862, 2576, 3706, 26041, 5071, 262, 5509, 13, 1375, 16555, 329, 607, 6639, 2802, 284, 307, 28557, 13, 7381, 330, 18117, 11, 607, 4601, 1625, 2081, 13, 3000, 4104, 11, 290, 661, 422, 1290, 290, 3

Enter the prompt for the story (Press Q to quit): q

Exiting program...
